In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd  /content/drive/MyDrive/Capstone2/
!ls

/content/drive/MyDrive/Capstone2
class-descriptions-boxable.csv	test
features			train-annotations-bbox.csv
image				train-images-boxable-with-rotation.csv
Inception.ipynb			train_process.ipynb
labels				Untitled0.ipynb
model				Untitled1.ipynb
preprocess_image.ipynb


In [ ]:
import os
import tensorflow
import tensorflow.compat.v1 as tf
from tensorflow.compat.v2.io import gfile
import tensorboard as tb
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder 
from sklearn.svm import SVC
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import pickle

In [ ]:
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [30]:
IMAGE_PATH ='./image'
TEST_PATH = './test'
MODEL_PATH ='./model'
OUTPUT_MODEL_NAME = 'classifier_oven_person.sav'
INCEPTION_MODEL = MODEL_PATH + '/tensorflow_inception_graph.pb'

In [ ]:
def get_data():
  print("[INFO] Start loading image...")
  images = os.listdir(IMAGE_PATH)
  images = [IMAGE_PATH + '/' + name for name in images if not name.startswith('.')]
  labels = ['Oven' if 'Oven' in image_name else 'Person' for image_name in images ]
  print("[INFO] Done loading image...")
  return images,labels

In [ ]:
def create_graph(path_model):
  with gfile.GFile(path_model,'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [ ]:
def extract_features(image_paths_array = []):
  """
    Extract bottleneck features
    Return: 2d array with a dimension of 2048 
  """
  features_dimension = 2048
  features = np.empty((len(image_paths_array),2048))
  with tf.Session() as sess:
    flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')
    for i,image_path in enumerate(image_paths_array):
      
      print('Processing {}/{}:{}...' .format(i+1,len(image_paths_array),image_path))
      fs = tf.io.gfile.get_filesystem(image_path)
      if not fs.exists(image_path):
        tf.logging.fatal('File does not exist',image_path)
      image_data = gfile.GFile(image_path, 'rb').read()
      feature = sess.run(flattened_tensor, {
        'DecodeJpeg/contents:0': image_data
      })
      features[i, :] = np.squeeze(feature)
  return features


In [ ]:
def train_svm_classifer(features, labels, model_output_path):
    """
    train_svm_classifer will train a SVM, saved the trained and SVM model and
    report the classification performance
    features: array of input features
    labels: array of labels associated with the input features
    model_output_path: path for storing the trained svm model
    """
    # save 20% of data for performance evaluation
    X_train, X_test, y_train, y_test = train_test_split(features, labels,shuffle = True, test_size=0.2)

    param = [
        {
            "kernel": ["linear"],
            "C": [1, 10, 100, 1000]
        },
        {
            "kernel": ["rbf"],
            "C": [1, 10, 100, 1000],
            "gamma": [1e-2, 1e-3, 1e-4, 1e-5]
        }
    ]

    # request probability estimation
    svm = SVC(probability=True)

    # 10-fold cross validation, use 4 thread as each fold and each parameter set can be train in parallel
    clf = GridSearchCV(svm, param,
            cv=10, n_jobs=4, verbose=3)

    clf.fit(X_train, y_train)
    
    if not os.path.exists(model_output_path):
        joblib.dump(clf.best_estimator_, model_output_path)
    else:
        print("Cannot save trained svm model to {0}.".format(model_output_path))

    print("\nBest parameters set:")
    print(clf.best_params_)

    y_predict=clf.predict(X_test)

    return y_predict,y_test,clf

In [ ]:
def predict_test_image(model):
  images = os.listdir(TEST_PATH)
  images = [TEST_PATH + '/' + name for name in images if not name.startswith('.')]
  true_labels = ['oven' if 'oven' in image_name.lower() else 'person' for image_name in images ]
  
  features = extract_features(images)
  y_pred = model.predict(features)
  print("True label of image: {} \n".format(true_labels))
  print("Predict label of image:{}".format(y_pred))
  

In [ ]:
#Get path from annotations file
img_file_paths,labels = get_data()


[INFO] Start loading image...
[INFO] Done loading image...


In [ ]:
#labels = le.fit_transform(labels)

In [ ]:
create_graph(INCEPTION_MODEL)

In [ ]:
features = extract_features(image_paths_array=img_file_paths)


Processing 1/1054:./image/Oven-43c822f0ab198276.jpg...
Processing 2/1054:./image/Oven-0c3c55d740f28eb3.jpg...
Processing 3/1054:./image/Oven-145f548d2fcc926c.jpg...
Processing 4/1054:./image/Oven-34104da91512a3dc.jpg...
Processing 5/1054:./image/Oven-3209c0aeb2ab6b62.jpg...
Processing 6/1054:./image/Oven-1469bd1746db6d33.jpg...
Processing 7/1054:./image/Oven-c2ee6842aa012389.jpg...
Processing 8/1054:./image/Oven-49f7e61e228d3311.jpg...
Processing 9/1054:./image/Oven-02d071c98c8615a4.jpg...
Processing 10/1054:./image/Oven-9a03191beeae10cc.jpg...
Processing 11/1054:./image/Oven-17083d5cea0b2bde.jpg...
Processing 12/1054:./image/Oven-64f48726c64d158b.jpg...
Processing 13/1054:./image/Oven-06553dad53cd2445.jpg...
Processing 14/1054:./image/Oven-691164e891ffe723.jpg...
Processing 15/1054:./image/Oven-22dcef85e8d20f4c.jpg...
Processing 16/1054:./image/Oven-0c9275268eaab6c2.jpg...
Processing 17/1054:./image/Oven-5b61fbd4d345a769.jpg...
Processing 18/1054:./image/Oven-5176110d7a45c84c.jpg...
P

In [ ]:
pickle.dump(features, open('features', 'wb'))
pickle.dump(labels, open('labels', 'wb'))

In [ ]:
features = pickle.load(open('features','rb'))
labels = pickle.load(open('labels','rb'))

In [ ]:
print("[INFO] Start classfication with SVM")
y_predict,y_test,clf = train_svm_classifer(features, labels, 'classifier_oven_person.sav')
print("[INFO] Done classfication with SVM")

[INFO] Start classfication with SVM
Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   28.0s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:  5.1min
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:  7.8min finished


Cannot save trained svm model to classifier_oven_person.sav.

Best parameters set:
{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
[INFO] Done classfication with SVM


In [ ]:
labels=sorted(list(set(labels)))
print("\n [INFO] Confusion matrix:")
print(confusion_matrix(y_test,y_predict))
print("Labels: {0}\n".format(",".join(labels)))
print("\nClassification report:")
print(classification_report(y_test, y_predict))


 [INFO] Confusion matrix:
[[ 88   2]
 [  0 121]]
Labels: Oven,Person


Classification report:
              precision    recall  f1-score   support

        Oven       1.00      0.98      0.99        90
      Person       0.98      1.00      0.99       121

    accuracy                           0.99       211
   macro avg       0.99      0.99      0.99       211
weighted avg       0.99      0.99      0.99       211



In [31]:
loaded_model = joblib.load(OUTPUT_MODEL_NAME)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-31-737c8e9cd193>", line 1, in <module>
    loaded_model = joblib.load(OUTPUT_MODEL_NAME)
  File "/usr/local/lib/python3.7/dist-packages/joblib/numpy_pickle.py", line 577, in load
    with open(filename, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: 'classifier_oven_person.sav'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-package

FileNotFoundError: ignored

In [ ]:
predict_test_image(model = loaded_model)

Processing 1/9:./test/Oven - 12321321312321.jpg...
Processing 2/9:./test/Wall-oven-hero2.jpg...
Processing 3/9:./test/oven - 2.jpeg...
Processing 4/9:./test/person_2.jpg...
Processing 5/9:./test/Luan.jpg...
Processing 6/9:./test/oven_3.jpg...
Processing 7/9:./test/best-air-fryer-toaster-oven.jpg...
Processing 8/9:./test/person_4.jpeg...
Processing 9/9:./test/person.jfif...
True label of image: ['oven', 'oven', 'oven', 'person', 'person', 'oven', 'oven', 'person', 'person'] 

Predict label of image:['Oven' 'Oven' 'Oven' 'Person' 'Person' 'Oven' 'Oven' 'Person' 'Person']
